# Reproducing in-text results 6 and 7

These results are variants on the scenario from Figure 2B, when the IAT is 3 minutes (170 arrivals/day) and the NCD nurse utilisation exceeds 100% (is at 123%). The variants are to:

* Assign administrative work to the staff nurse
* Have the staff nurse complete 10% of NCD checks

## Set up

In [1]:
# To run model
import PHC

# To import results and produce figures
from reproduction_helpers import process_results
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np

# To speed up run time
from multiprocessing import Pool

# Additional package to record runtime of this notebook
import time
start = time.time()

In [2]:
# Paths to save results to
output_folder = 'outputs'
txt6_path = os.path.join(output_folder, 'intext6.csv')
txt7_path = os.path.join(output_folder, 'intext7.csv')

## Run model with 10 replications

In [3]:
dict_list = [
    {
        'OPD_iat': 3,
        'rep_file': 'in67_base.xls',
    },
    {
        'OPD_iat': 3,
        'admin_ncd_to_staff': True,
        'rep_file': 'in6_base_admin.xls',
    },
    {
        'OPD_iat': 3,
        'opd_10p_ncd_to_staff': True,
        'rep_file': 'in7_base_appointment.xls',
    },
    {
        'OPD_iat': 3,
        'admin_ncd_to_staff': True,
        'opd_10p_ncd_to_staff': True,
        'rep_file': 'in7_base_admin_appointment.xls',
    }
]

In [4]:
# Append 's_' to all items
for i, d in enumerate(dict_list):
    dict_list[i] = {f's_{k}': v for k, v in d.items()}

In [5]:
# Wrapper function to allow input of dictionary with pool
def wrapper(d):
    return PHC.main(**d)

# Create a process pool that uses all CPUs
with Pool() as pool:
    # Run PHC.main() using each of inputs from config
    pool.map(wrapper, dict_list)

 No of replications done 0
 No of replications done 0
 No of replications done 0
 No of replications done 0
 No of replications done 1
 No of replications done 1
 No of replications done 1
 No of replications done 1
 No of replications done 2
 No of replications done 2
 No of replications done 2
 No of replications done 2
 No of replications done 3
 No of replications done 3
 No of replications done 3
 No of replications done 3
 No of replications done 4
 No of replications done 4
 No of replications done 4
 No of replications done 4
 No of replications done 5
 No of replications done 5
 No of replications done 5
 No of replications done 5
 No of replications done 6
 No of replications done 6
 No of replications done 6
 No of replications done 6
 No of replications done 7
 No of replications done 7
 No of replications done 7
 No of replications done 7
 No of replications done 8
 No of replications done 8
 No of replications done 8
 No of replications done 8
 No of replications done 9
 

## Run final scenario with 100 replications

This is due to discrepancy with paper, to confirm that increasing the replication number does not give us a result close to 71%.

In [6]:
PHC.main(s_OPD_iat=3,
         s_admin_ncd_to_staff=True,
         s_opd_10p_ncd_to_staff=True,
         s_replication=100,
         s_rep_file='in7_base_admin_appointment_100rep.xls')

 No of replications done 0
 No of replications done 1
 No of replications done 2
 No of replications done 3
 No of replications done 4
 No of replications done 5
 No of replications done 6
 No of replications done 7
 No of replications done 8
 No of replications done 9
 No of replications done 10
 No of replications done 11
 No of replications done 12
 No of replications done 13
 No of replications done 14
 No of replications done 15
 No of replications done 16
 No of replications done 17
 No of replications done 18
 No of replications done 19
 No of replications done 20
 No of replications done 21
 No of replications done 22
 No of replications done 23
 No of replications done 24
 No of replications done 25
 No of replications done 26
 No of replications done 27
 No of replications done 28
 No of replications done 29
 No of replications done 30
 No of replications done 31
 No of replications done 32
 No of replications done 33
 No of replications done 34
 No of replications done 35
 N

## Process results (from 10 replications)

In [7]:
# Process results
data = process_results([i['s_rep_file'] for i in dict_list], xls=True)

In [8]:
def get_util(df):
    '''
    Gets NCD utilisation and transforms the dataframe to aid readability

    Parameters:
    -----------
    df : pandas DataFrame
        Dataframe with mean results for different outputs

    Returns:
    --------
    util : pandas DataFrame
        Dataframe with the NCD utilisation in each scenario
    '''
    # Get NCD utilisation
    util = round(pd.DataFrame(df.loc['NCD occ']).T, 3)

    # Rename index and row
    util = util.rename_axis('Output')
    util = util.rename({'NCD occ': 'NCD nurse utilisation'})

    # Rename columns
    util = util.rename({
        'in67_base': 'Normal',
        'in6_base_admin': 'Admin from NCD to staff nurse',
        'in7_base_appointment': '10% OPD appointments from NCD to staff nurse',
        'in7_base_admin_appointment': (
            'Admin and 10% OPD appointments from NCD to staff nurse')}, axis=1)

    return util

### Get in-text result 6

In [9]:
txt6 = get_util(data[['in67_base', 'in6_base_admin']])

# Save and display results
txt6.to_csv(txt6_path)
txt6

,Normal,Admin from NCD to staff nurse
Output,,
NCD nurse utilisation,1.247,0.984


### Get in-text result 7

In [10]:
txt7 = get_util(data)

# Save and display results
txt7.to_csv(txt7_path)
txt7

,Normal,Admin from NCD to staff nurse,10% OPD appointments from NCD to staff nurse,Admin and 10% OPD appointments from NCD to staff nurse
Output,,,,
NCD nurse utilisation,1.247,0.984,1.07,0.828


## Get in-text result 7 scenario from 100 replications

In [11]:
res_100 = process_results(['in7_base_admin_appointment_100rep.xls'], xls=True)
res_100.loc['NCD occ']

in7_base_admin_appointment_100rep    0.827133
Name: NCD occ, dtype: float64

## Run model with 20% 

## Run time

In [12]:
# Find run time in seconds
end = time.time()
runtime = round(end-start)

# Display converted to minutes and seconds
print(f'Notebook run time: {runtime//60}m {runtime%60}s')

Notebook run time: 55m 47s
